In [1]:
from email.parser import Parser
import re
import time
from datetime import datetime, timezone, timedelta

def date_to_dt(date):
    def to_dt(tms):
        def tz():
            return timezone(timedelta(seconds=tms.tm_gmtoff))
        return datetime(tms.tm_year, tms.tm_mon, tms.tm_mday, 
                      tms.tm_hour, tms.tm_min, tms.tm_sec, 
                      tzinfo=tz())
    return to_dt(time.strptime(date[:-6], '%a, %d %b %Y %H:%M:%S %z'))

In [2]:
# Q1: replace pass with your code
def extract_email_network(rdd):
    email_regex = '(^[a-zA-Z0-9.!#$%&\'*+-/=?^_`{|}~]*[@]([\w]+[.])*)(\w*?[^0-9.][\w]*)'
    
    def gen_seq_from_csv_string(s):
        m = re.compile('([^\s,]+)').search(s, 0)
        while m:
            yield m.group(1)
            m = re.compile('([^\s,]+)').search(s, m.end())

    # Replace the right-hand side of the lambda with your code
    valid_email = lambda s: True if re.compile(email_regex).fullmatch(s) != None else False

    concat_csv_strings = lambda s1, s2: (s1 + ',' + s2) if (s1 != '' and s2 != '') else blank_csv_string(s1,s2)
    blank_csv_string = lambda s1, s2 : s1 if s1 != '' else s2
    # concat_csv_strings = lambda s1, s2: print("why")

    val_by_vec = lambda x, t, u: ((x,i, u) for i in t)
    not_self_loop = lambda t: True if t[0]!=t[1] else False

    RDD1 = rdd.map(lambda x : Parser().parsestr(x))
    rdd2 = RDD1.map(lambda x : (x.get('From'), x.get('To'), x.get('Cc'), x.get('Bcc'), x.get('Date')))

    rdd3 = rdd2.filter(lambda x: valid_email(x[0]) )
    rdd4 = rdd3.map(lambda x : (x[0], concat_csv_strings(x[1] if x[1] != None else '',concat_csv_strings(x[2] if x[2] != None else '',x[3] if x[3] != None else '')), date_to_dt(x[4])))

    rdd5 = rdd4.map(lambda x : (x[0], tuple(gen_seq_from_csv_string(x[1])), x[2]))
    rdd6 = rdd5.flatMap(lambda x :(val_by_vec(x[0],x[1],x[2])))
    rdd7 = rdd6.filter(lambda x : valid_email(x[1]))
    rdd8 = rdd7.filter(lambda x: not_self_loop(x))
    rdd_domain = rdd8.filter(lambda x : (x[0].endswith('enron.com') and x[1].endswith('enron.com')))
    rdd9 = rdd_domain.distinct()
    return rdd9

# Q2: replace pass with your code
def convert_to_weighted_network(rdd, drange=None):
    if (drange != None):
        rdd = rdd.filter(lambda x : (x[2] > drange[0]) and (x[2] < drange[1]))
    
#     rdd10 = rdd.distinct().count()
    rdd10 = rdd.map(lambda x : ((x[0], x[1]), x[2]))
    rdd11 = rdd10.groupByKey().mapValues(len)
    rdd12 = rdd11.map(lambda x : (x[0][0], x[0][1], x[1]))
    return rdd12
    #     pass

# Q3.1: replace pass with your code
def get_out_degrees(rdd):
    rdd13 = rdd.map(lambda x : (x[0], x[2]))
    rdd14 = rdd.map(lambda x: (x[1], 0))
    rdd15 = rdd13.union(rdd14)
    
    rdd16 = rdd15.groupByKey().mapValues(sum)
    rdd17 = rdd16.map(lambda x : (x[1], x[0])).sortBy(lambda x : (x[0],x[1]),ascending = False)
    return rdd17
#     pass

# Q3.2: replace pass with your code         
def get_in_degrees(rdd):
    rdd18 = rdd.map(lambda x : (x[1], x[2]))
    rdd19 = rdd.map(lambda x: (x[0], 0))
    rdd20 = rdd19.union(rdd18)
    
    rdd21 = rdd20.groupByKey().mapValues(sum)
    rdd22 = rdd21.map(lambda x : (x[1], x[0])).sortBy(lambda x : (x[0],x[1]),ascending = False)
    return rdd22
#     pass

# Q4.1           
def get_out_degree_dist(rdd):
    out_degree_rdd = get_out_degrees(rdd)
    rdd23 = out_degree_rdd.groupByKey().mapValues(len).sortBy(lambda x : (x[0],x[1]),ascending = True)
    return rdd23


# Q4.2
def get_in_degree_dist(rdd):
    in_degree_rdd = get_in_degrees(rdd)
    rdd24 = in_degree_rdd.groupByKey().mapValues(len).sortBy(lambda x : (x[0],x[1]),ascending = True)
    return rdd24